In [33]:
import numpy as np 
import os
import re
from tqdm import tnrange
import matplotlib.pyplot as plt
import time

In [34]:
sbox = [
    # 0    1    2    3    4    5    6    7    8    9    a    b    c    d    e    f 
    0x63,0x7c,0x77,0x7b,0xf2,0x6b,0x6f,0xc5,0x30,0x01,0x67,0x2b,0xfe,0xd7,0xab,0x76, # 0
    0xca,0x82,0xc9,0x7d,0xfa,0x59,0x47,0xf0,0xad,0xd4,0xa2,0xaf,0x9c,0xa4,0x72,0xc0, # 1
    0xb7,0xfd,0x93,0x26,0x36,0x3f,0xf7,0xcc,0x34,0xa5,0xe5,0xf1,0x71,0xd8,0x31,0x15, # 2
    0x04,0xc7,0x23,0xc3,0x18,0x96,0x05,0x9a,0x07,0x12,0x80,0xe2,0xeb,0x27,0xb2,0x75, # 3
    0x09,0x83,0x2c,0x1a,0x1b,0x6e,0x5a,0xa0,0x52,0x3b,0xd6,0xb3,0x29,0xe3,0x2f,0x84, # 4
    0x53,0xd1,0x00,0xed,0x20,0xfc,0xb1,0x5b,0x6a,0xcb,0xbe,0x39,0x4a,0x4c,0x58,0xcf, # 5
    0xd0,0xef,0xaa,0xfb,0x43,0x4d,0x33,0x85,0x45,0xf9,0x02,0x7f,0x50,0x3c,0x9f,0xa8, # 6
    0x51,0xa3,0x40,0x8f,0x92,0x9d,0x38,0xf5,0xbc,0xb6,0xda,0x21,0x10,0xff,0xf3,0xd2, # 7
    0xcd,0x0c,0x13,0xec,0x5f,0x97,0x44,0x17,0xc4,0xa7,0x7e,0x3d,0x64,0x5d,0x19,0x73, # 8
    0x60,0x81,0x4f,0xdc,0x22,0x2a,0x90,0x88,0x46,0xee,0xb8,0x14,0xde,0x5e,0x0b,0xdb, # 9
    0xe0,0x32,0x3a,0x0a,0x49,0x06,0x24,0x5c,0xc2,0xd3,0xac,0x62,0x91,0x95,0xe4,0x79, # a
    0xe7,0xc8,0x37,0x6d,0x8d,0xd5,0x4e,0xa9,0x6c,0x56,0xf4,0xea,0x65,0x7a,0xae,0x08, # b
    0xba,0x78,0x25,0x2e,0x1c,0xa6,0xb4,0xc6,0xe8,0xdd,0x74,0x1f,0x4b,0xbd,0x8b,0x8a, # c
    0x70,0x3e,0xb5,0x66,0x48,0x03,0xf6,0x0e,0x61,0x35,0x57,0xb9,0x86,0xc1,0x1d,0x9e, # d
    0xe1,0xf8,0x98,0x11,0x69,0xd9,0x8e,0x94,0x9b,0x1e,0x87,0xe9,0xce,0x55,0x28,0xdf, # e
    0x8c,0xa1,0x89,0x0d,0xbf,0xe6,0x42,0x68,0x41,0x99,0x2d,0x0f,0xb0,0x54,0xbb,0x16  # f
]

sbox_2d = np.reshape(sbox, (16, 16))

N = 256 

def calc_hamming_weight(n):
    return bin(n).count("1")

bit_vals    = np.arange(0,256)
HW          = [calc_hamming_weight(k) for k in bit_vals]



def int_2_hex_string(input):
    return (("0x%0.2X" % input).split('x')[-1])


def int_2_hex_string_32(input):
    return (("0x%0.32X" % input).split('x')[-1])

def sub_bytes(next):
    s_next = int_2_hex_string(next)
    return (sbox_2d[int(s_next[1],16)][int(s_next[0],16)])


def add_round_key(key, next):
    return (key ^ next)


def forward_lookup(next, key):
    return(add_round_key(sub_bytes(next), key))


def generate_loopup_table():
    lookup_table = np.zeros((N,N))
    for ct_prev in range(N):
        for key in range(N):
            lookup_table[ct_prev,key] = forward_lookup(ct_prev, key)

    return lookup_table.astype(int)


def inverse_lookup(key_8, ct_8):
    global lookup_table
    column = lookup_table[:, key_8]

    return np.where(column == ct_8)[0][0] 


def get_unarranged_prev_ct(key, ct):
    prev_ct_arr_unarranged = []
    for i in range(16):
        key_8_int = int( (key[i*2: (i*2)+2]) , 16 )
        ct_8_int  = int( (ct[i*2: (i*2)+2]) , 16 )
        prev_ct_arr_unarranged.append( int_2_hex_string( inverse_lookup(key_8_int, ct_8_int) )[::-1] )

    return prev_ct_arr_unarranged


def get_mapping():
    key     = 'd014f9a8c9ee2589e13f0cc8b6630ca6'
    ct      = '3ad77bb40d7a3660a89ecaf32466ef97'
    prev_ct = 'bb36c7eb88334d49a4e7112e74f182c4'

    prev_ct_arr_unarranged  = get_unarranged_prev_ct(key, ct)
    prev_ct_arr_arranged    = re.findall('..',prev_ct.upper())

    mapping = []
    
    for i in range(16):
        mapping.append(prev_ct_arr_arranged.index(prev_ct_arr_unarranged[i]))
    return mapping

# def rearrange_prev_ct(unarranged):
#     mapping = [0, 5, 10, 15, 4, 9, 14, 3, 8, 13, 2, 7, 12, 1, 6, 11]
#     arranged = ""

#     for i in range(16):
#         arranged = arranged + unarranged[mapping.index(i)] 

#     return arranged'


lookup_table = generate_loopup_table()


In [35]:
def getCipher(dir="Simulations/ct"):

    line =[]
    cypher=[]
    names=[]

    directory = os.fsencode(dir)

    for file in os.listdir(directory):

        filename = os.fsdecode(file)

        with open(os.path.join(dir,filename),'r') as f:
            names.append(filename)
            line=(f.readlines()) #loading cypher
            cypher.append(line)
        
    cypher_floats=[]
    for k in cypher:
        container=[]
        for j in k:
            temp=j.strip()
            temp2="0x"+temp
            temp3=int(temp2,16)
            container.append(temp3)
        cypher_floats.append(container)
                
    return names,cypher_floats

def get_load_order(names_list):
    order=[]
    for k in names_list:
        order.append(int((k.split(".")[0]).split("_")[-1]))
    return order

def getPower(dir_="Simulations/power"):
    power_file=[]
    power=[]
    names=[]

    directory = os.fsencode(dir_)

    for file in os.listdir(directory):

        filename = os.fsdecode(file)

        if filename.endswith("txt"):
            names.append(filename)
            with open(os.path.join(dir_,filename),'r') as f:
                power_file=(f.readlines()) #loading power
                power.append(power_file)
    
    power_floats=[]
    for k in power:
        container=[]
        for j in k:
            temp=j.strip()
            temp2=float(temp)
            container.append(temp2)
        power_floats.append(container)

    return names,power_floats

def get_r10_keys(dir_="../Simulations/"):
    keys=[]
    keys_final=[]

    directory = os.fsencode(dir_)

    for file in os.listdir(dir_):

        filename = os.fsdecode(file)

        if filename.endswith("round10_key.txt"):
            with open(os.path.join(dir_,filename),'r') as f:
                key_line=(f.readlines()) #loading power
                keys.append(key_line)
    keys=keys[0]
    
    for k in keys: 
        temp0=k.split(" ")[-1]
        temp=temp0.strip()
        temp2="0x"+temp
        temp3=int(temp2,16)
        keys_final.append(temp3)
        
    return keys_final

### Getting CT

In [36]:
cipher_names,ciphers=getCipher("../Simulations/ct")
# print(get_load_order(cipher_names))
# print(len(ciphers[1]))

### Getting Power

In [37]:
power_names,power_traces=getPower("../Simulations/power")
# print(get_load_order(power_names))
# print(power_traces[3][3516])


### Get Key and Reorder Key

In [38]:
#Getting the Keys
r10_keys = get_r10_keys()

#Reordering to match

#get order in terms of indeces
order = [k-1 for k in get_load_order(cipher_names)]
#print(order)

#reordering the keys
r10_keys = [r10_keys[i] for i in order]

In [39]:
target_run = 0

# Get only useful data
pwr_trc = power_traces[target_run]
ct      = ciphers[target_run]

# print(pwr_trc[0])
# print(int_2_hex_string_32(ct[0]))

## CPA

The actual algorithm is as follows for datasets $X$ and $Y$ of length $N$, with means of $\bar{X}$ and $\bar{Y}$, respectively:

$$r = \frac{cov(X, Y)}{\sigma_X \sigma_Y}$$

$cov(X, Y)$ is the covariance of `X` and `Y` and can be calculated as follows:

$$cov(X, Y) = \sum_{n=1}^{N}[(Y_n - \bar{Y})(X_n - \bar{X})]$$

$\sigma_X$ and $\sigma_Y$ are the standard deviation of the two datasets. This value can be calculated with the following equation:

$$\sigma_X = \sqrt{\sum_{n=1}^{N}(X_n - \bar{X})^2}$$



In [40]:
def mean(X):
    return np.mean(X,axis=0)

def std_dev(X, X_bar):
    return np.sqrt(np.sum((X-X_bar)**2,axis=0))

def cov(X, X_bar, Y, Y_bar):
    return np.sum((X-X_bar)*(Y-Y_bar),axis=0)

In [41]:
NUMBER_OF_TRACES_USED       = 5000
NUMBER_KEYS                 = 256
NUMBER_BYTES_UNDER_ATTACK   = 1

pwr_trc_bar = mean(np.array(pwr_trc[0:NUMBER_OF_TRACES_USED]))                       # mean
std_pwr_trc = std_dev( np.array(pwr_trc[0:NUMBER_OF_TRACES_USED]), pwr_trc_bar )     # std_dev

#print(std_pwr_trc)
# print("Pwer trace mean: ", pwr_trc_bar)
# print("std_dev pwr_trc: ", std_pwr_trc)

bestguess = []                                              # key byte guesses here

all_hws = []
all_corr_coeffs = []

for bnum in tnrange(0,NUMBER_BYTES_UNDER_ATTACK):
    corr_coeffs = [0] * NUMBER_KEYS
    for kguess in range(0, NUMBER_KEYS):
        
        #Repeating the key byte guessing process from the last loop
        hws = []
        ct_8_arr = []
        prev_ct_8_arr = []

        start = bnum * 2 
        stop  = start + 2

        for i in range(NUMBER_OF_TRACES_USED):
            ct_8_arr.append( int_2_hex_string_32( ct[i])[start:stop] )

        #print(ct_8_arr)
        for i in range(NUMBER_OF_TRACES_USED):
            temp = int_2_hex_string( inverse_lookup(kguess, int( ct_8_arr[i],16)) ) 
            prev_ct_8_arr.append(temp) 
        
        #print(prev_ct_8_arr)

        for i in range(NUMBER_OF_TRACES_USED):
            op1 = int(prev_ct_8_arr[i],16)
            op2 = int(ct_8_arr[i],16)
            res = op1 ^ op2
            hws.append(HW[res])  


        all_hws.append(hws)

        hws                 = np.array(hws)
        hws_bar             = mean(hws)
        std_hws             = std_dev(hws,hws_bar)
        corr_coeff          = (cov(hws,hws_bar,pwr_trc[0:NUMBER_OF_TRACES_USED],pwr_trc_bar))/(std_pwr_trc*std_hws)
        corr_coeffs[kguess] = corr_coeff


    abs_corr_coeffs = [ abs(k) for k in corr_coeffs ]
    all_corr_coeffs.append(abs_corr_coeffs)


#print(all_hws)



C:\Users\ub352\AppData\Local\Temp/ipykernel_14076/2400447786.py:17: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for bnum in tnrange(0,NUMBER_BYTES_UNDER_ATTACK):


  0%|          | 0/1 [00:00<?, ?it/s]

In [42]:
trgt_bnum = 0
no_of_top_coeffs = 10

# Get index of top {no_of_top_coeffs} corr_coeffs
top_n_coeffs_idx = sorted(range(len(all_corr_coeffs[trgt_bnum])), 
                        key = lambda sub: all_corr_coeffs[trgt_bnum][sub])[-no_of_top_coeffs:]

# Sort the index of top {no_of_top_coeffs} correlation coefficients based on their corr_coeff value
top_n_coeffs_idx.reverse()

top_n_correlations = [all_corr_coeffs[trgt_bnum][k] for k in top_n_coeffs_idx]


print("Top " + str(no_of_top_coeffs) + " corr coeffs for byte " + str(trgt_bnum) + " : ", top_n_correlations)
print("Index of top " + str(no_of_top_coeffs) + " corr coeffs: ", top_n_coeffs_idx)

print(max(all_corr_coeffs[trgt_bnum])) 


Top 10 corr coeffs for byte 0 :  [0.06055651616695104, 0.053134992067480416, 0.052896548833466124, 0.05015555167322295, 0.04913209313464251, 0.048480629804298965, 0.04706269304493913, 0.046861270229958917, 0.04526942568972423, 0.04506696750953339]
Index of top 10 corr coeffs:  [14, 195, 217, 62, 91, 201, 165, 240, 4, 86]
0.06055651616695104


In [43]:
found_key_hex = [int_2_hex_string(k) for k in top_n_coeffs_idx] 

#found_key = "".join(keyy_hex)

print("Top " + str(no_of_top_coeffs) + " possible key bytes: ", found_key_hex) 
#print("Actual key bytes: ", int_2_hex_string_32(r10_keys[target_run])) 

Top 10 possible key bytes:  ['0E', 'C3', 'D9', '3E', '5B', 'C9', 'A5', 'F0', '04', '56']
